# Trotterized Adiabatic State Preparation (TASP)in Qiskit

In this notebook, we will demonstrate how to use the Trotterized Adiabatic State Preparation ansatz to solve for the ground state of the Hydrogen molecule and discuss its advantages over the more traditional UCCSD ansatz.

The Variational Quantum Eigensolver is typically used in connection with the Unitary Coupled Cluster Single & Double (UCCSD) variational form. This ansatz takes the form

\begin{align}
|\psi (\theta)> &= e^{i\left[T(\theta)-T^\dagger(\theta)\right]} |\text{HF}>  \\
T(\theta) &= \theta^{ij} a^\dagger_i a_j + \theta^{ijkl} a^\dagger_i a^\dagger_j a_k ,
\end{align}

where HF denotes the Hartree-Fock initial state (i.e. the best Slater determinant). This functionality is already implemented in Qiskit in a straightforward way:



In [22]:
import numpy as np
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
from qiskit.aqua.components.variational_forms import TASP
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit.aqua.components.optimizers import COBYLA
from qiskit import BasicAer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance


#Classical part: set up molecular data, map to WeightedPauliOperator and find exact solution
dist=0.735
driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(dist), unit=UnitsType.ANGSTROM, 
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()
repulsion_energy = molecule.nuclear_repulsion_energy
num_spin_orbitals=molecule.num_orbitals*2

map_type='jordan_wigner'

ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)

exact_solution = ExactEigensolver(qubitOp).run()['energy']
exact_solution=exact_solution+repulsion_energy


#Set up initial state and UCCSD variational form with standard parameters
HF_state=HartreeFock(qubitOp.num_qubits, num_spin_orbitals, num_particles, map_type)
varform_UCCSD = UCCSD(qubitOp.num_qubits, depth=1, num_orbitals=num_spin_orbitals, num_particles=num_particles,
                 active_occupied=None, active_unoccupied=None, initial_state=HF_state,
                 qubit_mapping='jordan_wigner', two_qubit_reduction=False, num_time_slices=1,
                 shallow_circuit_concat=True, z2_symmetries=None)




#Run VQE using UCCSD variational form and compare with exact solution
print('Molecule H2 at bond length ' + str(dist))
print('Running VQE with UCCSD. This may take a few minutes. Some intermediary energies while you wait:')
opt_iter=15
optimizer = COBYLA(maxiter=opt_iter)
n_shots=1000
instance=QuantumInstance(backend=BasicAer.get_backend("qasm_simulator"), shots=n_shots)
vqe_instance = VQE(qubitOp, varform_UCCSD, optimizer=optimizer)
result=vqe_instance.run(instance)
energy=result['energy']+repulsion_energy 
print('Optimal energy is ' + str(energy) +'. Exact result is ' + str(exact_solution))



two_qubit_reduction only works with parity qubit mapping but you have jordan_wigner. We switch two_qubit_reduction to False.


Molecule H2 at bond length 0.735
Running VQE with UCCSD. This may take a few minutes. Some intermediary energies while you wait:
-1.8340730920064368
-1.2879876334010514
-1.2869690642127465
-0.5566849913388404
-1.2220256930442654
-1.6537314515250268
-1.8174333760071821
-1.8354617673090807
-1.6971453724502985
-1.8531740050480021
-1.8062412535175663
-1.8332978984146708
-1.8352722842777764
-1.8547615100882051
-1.835785814794625
Optimal energy is -1.1347925156392256. Exact result is -1.137306035753399


# The idea behind TASP

Note that during the optimization, in this case the COBYLA optimizer has 3 parameters to optimize over, $\theta^{1,3}$, $\theta^{2,4}$ and $\theta^{1,2,3,4}$ (taking into account orbitals with different spin). Convergence of the classical optimizer is no problem for this modest number of parameters. However, it is clear that for a large number $M$ of molecular orbitals, the number of parameters grows as $\mathcal{O} \left( M^4 \right)$, until at some point, the classical optimization becomes the bottleneck of the whole procedure.

To combat this problem, Wecker et al. [proposed](https://journals.aps.org/pra/pdf/10.1103/PhysRevA.92.042303) the Trotterized Adiabatic State Preparation ansatz. Splitting the Quantum Chemistry Hamiltonian as follows

\begin{align}
H &= \sum_{pq} h_{pq} a^\dagger_p a_q + \sum_{pqrq} h_{pqrs} a^\dagger_p a^\dagger_q a_r a_s \\ 
&= \underbrace{\sum_{p} h_{pp} a^\dagger_p a_p + \sum_{pq} h_{pqqp} a^\dagger_p a^\dagger_q a_q a_p}_{H_\text{diag}} + \underbrace{\sum_{p \neq q} h_{pq} a^\dagger_p a_q + \sum_{prq} h_{prrq} a^\dagger_p a^\dagger_r a_r a_q}_{H_\text{hop}} + \underbrace{\sum_{p \neq q \neq r \neq s} h_{pqrs} a^\dagger_p a^\dagger_q a_r a_s}_{H_\text{ex}}
\end{align}

and defining a depth $S$ we can define the following ansatz with $3S$ parameters:

\begin{align}
|\psi(\theta)> = \prod_{b=1}^S \exp \left(i \frac{\theta_\text{ex}^b}{2} H_\text{ex}\right) \exp \left(i \frac{\theta_\text{hop}^b}{2} H_\text{hop}\right) \exp \left(i \theta_\text{diag}^b H_\text{diag}\right) \exp \left(i \frac{\theta_\text{hop}^b}{2} H_\text{hop}\right) \exp \left(i \frac{\theta_\text{ex}^b}{2} H_\text{ex}\right)
\end{align}

Due to the advantageous splitting of the Hamiltonian, this ansatz is expected to not merely reduce the number of parameters but also lead to shallower circuits for large $M$, as long as we do not need to scale $S$ too badly to ensure the accuracy of the solution.

In [20]:
#Split the Hamiltonian into the abovementioned pieces
diag_one_body=np.diag(np.diagonal(molecule.one_body_integrals))
offdiag_one_body=molecule.one_body_integrals-diag_one_body

diag_two_body=np.zeros((num_spin_orbitals,num_spin_orbitals,num_spin_orbitals,num_spin_orbitals))
hop_two_body=np.zeros((num_spin_orbitals,num_spin_orbitals,num_spin_orbitals,num_spin_orbitals))
ex_two_body=np.zeros((num_spin_orbitals,num_spin_orbitals,num_spin_orbitals,num_spin_orbitals))

for i1 in range(num_spin_orbitals):
    for i2 in range(num_spin_orbitals):
        for i3 in range(num_spin_orbitals):
            for i4 in range(num_spin_orbitals):
                if i2==i3:
                    if i1==i4:
                        diag_two_body[i1,i2,i3,i4]=molecule.two_body_integrals[i1,i2,i3,i4]
                    else:
                        hop_two_body[i1,i2,i3,i4]=molecule.two_body_integrals[i1,i2,i3,i4]
                else:
                    ex_two_body[i1,i2,i3,i4]=molecule.two_body_integrals[i1,i2,i3,i4]
                    
H_diag=FermionicOperator(h1=diag_one_body,h2=diag_two_body)
H_hop=FermionicOperator(h1=offdiag_one_body,h2=hop_two_body)
H_ex=FermionicOperator(h1=np.zeros((num_spin_orbitals,num_spin_orbitals)),h2=ex_two_body)
H_diag=H_diag.mapping(map_type=map_type, threshold=0.00000001)
H_hop=H_hop.mapping(map_type=map_type, threshold=0.00000001)
H_ex=H_ex.mapping(map_type=map_type, threshold=0.00000001)
h_list=[H_ex, H_hop, H_diag]


#Define TASP variational form (the TASP class file is attached to original comment)
S=1
varform_TASP=TASP(num_spin_orbitals, depth=S, h_list=h_list, initial_state=HF_state)

print('Running VQE with TASP. This may take a few minutes. Some intermediary energies while you wait:')
vqe_instance = VQE(qubitOp, varform_TASP, optimizer=optimizer)
result=vqe_instance.run(instance)
energy=result['energy']+repulsion_energy 

print('Optimal energy is ' + str(energy) +'. Exact result is ' + str(exact_solution))

Running VQE with TASP. This may take a few minutes. Some intermediary energies while you wait:
-1.6570608288349202
-1.5452847937072123
-1.6856951200211077
-1.5745302406490531
-1.835986215366027
-1.8261315469120947
-1.829466019292867
-1.8023829890667007
-1.8482223566882925
-1.8282832936133415
-1.8475718940865262
-1.852784492075226
-1.8500144652836132
-1.84952773047911
-1.8295759125889228
Optimal energy is -1.1328154976262463. Exact result is -1.1373060357533973


Let us compare the two ansatz circuits in terms of number of parameters, gate depth and gate count:

In [21]:
n_param_UCCSD=varform_UCCSD.num_parameters
qc_UCCSD=varform_UCCSD.construct_circuit(np.random.rand(n_param_UCCSD))
print('Number of parameters for UCCSD:', n_param_UCCSD)
print('Depth for UCCSD:', qc_UCCSD.depth())
print('Gate counts:', qc_UCCSD.count_ops())

n_param_TASP=varform_TASP.num_parameters
qc_TASP=varform_TASP.construct_circuit(np.random.rand(n_param_TASP))
print('Number of parameters for TASP:', n_param_TASP)
print('Depth for TASP:', qc_TASP.depth())
print('Gate counts:', qc_TASP.count_ops())


Number of parameters for UCCSD: 3
Depth for UCCSD: 93
Gate counts: OrderedDict([('cx', 56), ('u3', 42), ('u2', 40), ('u1', 12), ('barrier', 3)])
Number of parameters for TASP: 3
Depth for TASP: 119
Gate counts: OrderedDict([('cx', 80), ('u1', 40), ('u3', 34), ('u2', 32), ('barrier', 4)])


We see that for a simple molecule like H2 in a minimal basis set, UCCSD performs better. Try replacing one of the Hydrogen atoms above with Lithium ('Li') and see what happens (if you have time ;-).


Note that the TASP variational form can be used for any list of operators, independant of the domain of the problem.